In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install selenium
!apt-get update

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,424 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelea

In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/nlp' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 120 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [ ]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.26.1


In [ ]:
chrome_path = "/content/drive/MyDrive/nlp/chromedriver"

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
#chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0900&dtlGnrCode=GN0901#params%5BgnrCode%5D=GN0900&params%5BdtlGnrCode%5D=GN0901&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1'
driver.get(url)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

song_ids = []

title_elements = driver.find_elements(By.CLASS_NAME, 'ellipsis.rank01')

for element in title_elements:
    # songId가 포함된 href 속성 추출
    link_element = element.find_element(By.TAG_NAME, 'a')
    href = link_element.get_attribute('href')

    # songId 추출 (두 번째 숫자만)
    song_id_match = re.search(r"playSong\('\d+',(\d+)\)", href)
    if song_id_match:
        song_ids.append(song_id_match.group(1))


In [ ]:
def craw_lyrics():
    try:
        artist = driver.find_elements(By.CLASS_NAME, 'artist')
        song_name = driver.find_elements(By.CLASS_NAME, 'song_name')
        lyric = driver.find_elements(By.CLASS_NAME, 'lyric')
        driver.implicitly_wait(60)

        lyrics = lyric[0].text.split('\n')

        artists.append(artist[0].text)
        song_names.append(song_name[0].text)
        ly.append(lyrics)
    except:
        return

In [ ]:
artists = []
song_names = []
ly = []

for i in song_ids:
      driver.get('https://www.melon.com/song/detail.htm?songId={song_num}'.format(song_num=i))
      craw_lyrics()

df = pd.DataFrame(artists, columns=['artist'])
df['song_name'] = song_names
df['lyric'] = ly

# 1~600 위 크롤링

In [ ]:
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller

# ChromeDriver 및 ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # GUI 비활성화
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

# WebDriver 초기화
driver = webdriver.Chrome(options=chrome_options)

all_artists = []
all_song_names = []
all_lyrics = []

def craw_lyrics():
    try:
        # 가수, 노래 제목, 가사 찾기
        artist = driver.find_element(By.CLASS_NAME, 'artist').text
        song_name = driver.find_element(By.CLASS_NAME, 'song_name').text
        lyric_element = driver.find_element(By.CLASS_NAME, 'lyric')

        # 가사는 줄바꿈 기준으로 분리
        lyrics = lyric_element.text.split('\n')

        # 각각 리스트에 추가
        all_artists.append(artist)
        all_song_names.append(song_name)
        all_lyrics.append(lyrics)
    except:
        # 가사가 없는 경우 "가사 없음"으로 처리
        all_artists.append(artist)
        all_song_names.append(song_name)
        all_lyrics.append("가사 없음")

for page_index in range(1, 552, 50):
    # URL에 페이지 인덱스 적용
    url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0900&dtlGnrCode=GN0901&po=pageObj&startIndex=1#params%5BgnrCode%5D=GN0900&params%5BdtlGnrCode%5D=GN0901&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={page_index}'
    driver.get(url)
    time.sleep(2)  # 로딩 기다리기 위해 시간 추가

    # 현재 페이지의 모든 songId 수집
    song_ids = []
    title_elements = driver.find_elements(By.CLASS_NAME, 'ellipsis.rank01')

    for element in title_elements:
        # href 속성에서 songId 추출
        link_element = element.find_element(By.TAG_NAME, 'a')
        href = link_element.get_attribute('href')
        song_id_match = re.search(r"playSong\('\d+',(\d+)\)", href)
        if song_id_match:
            song_ids.append(song_id_match.group(1))

    # 각 songId의 세부 페이지로 이동하여 가사 수집
    for song_id in song_ids:
        driver.get(f'https://www.melon.com/song/detail.htm?songId={song_id}')
        time.sleep(1)  # 로딩 위해 시간 추가
        craw_lyrics()

# DataFrame 생성
df = pd.DataFrame({
    'artist': all_artists,
    'song_name': all_song_names,
    'lyric': all_lyrics
})

# 결과 확인
print(df)

# WebDriver 종료
driver.quit()


                           artist                          song_name  \
0                    Charlie Puth      I Don't Think That I Like Her   
1    The Kid LAROI, Justin Bieber                               STAY   
2                    Charlie Puth                        Dangerously   
3                            Lauv  Steal The Show (From "Elemental")   
4                      Anne-Marie                               2002   
..                            ...                                ...   
595                 Ariana Grande                 Snow In California   
596                          HRVY               I Wish You Were Here   
597             Pharrell Williams       Happy (From Despicable Me 2)   
598                  Alex Sampson                         Wallflower   
599                 Billie Eilish                         WILDFLOWER   

                                                 lyric  
0    [Get her name and get her number, Find out all...  
1    [I do the same t

In [ ]:
df

,artist,song_name,lyric
0,Charlie Puth,I Don't Think That I Like Her,"[Get her name and get her number, Find out all..."
1,"The Kid LAROI, Justin Bieber",STAY,"[I do the same thing I told you, that I never ..."
2,Charlie Puth,Dangerously,"[This is gonna hurt, but I blame myself first,..."
3,Lauv,"Steal The Show (From ""Elemental"")","[Started out on a one-way train, Always knew w..."
4,Anne-Marie,2002,"[I will always remember, The day you kissed my..."
...,...,...,...
595,Ariana Grande,Snow In California,"[Dear Santa, It's me Ariana, I know it's been ..."
596,HRVY,I Wish You Were Here,"[I wish you were here, Every time you disappea..."
597,Pharrell Williams,Happy (From Despicable Me 2),"[It might seem crazy what, I'm about to say, S..."
598,Alex Sampson,Wallflower,"[I don't know, baby, How something so wonderfu..."


In [ ]:
df.to_excel("TOP600.xlsx")

In [ ]:
df.to_csv("TOP600.csv")